In [163]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import tensorflow as tf
from keras.layers import Dense, Activation, Input, Dropout
from keras import Sequential

from sklearn.model_selection import train_test_split as tts, StratifiedKFold as skf

In [164]:
data = pd.read_csv('../data/ppm.csv')

X = data.Year
Y = data.PPM

In [165]:
Y.head()

0    315.98
1    316.91
2    317.64
3    318.45
4    318.99
Name: PPM, dtype: float64

In [166]:
model = Sequential(
    [
        # Input(shape=(47,)),
        Dense(units=4096, activation='relu', input_shape=(1,)),
        # Dense(units=4096, activation='relu'),
        Dense(units=2048, activation='relu'),
        Dense(units=1024, activation='relu'),
        Dense(units=512, activation='relu'),
        Dense(units=256, activation='relu'),
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=32, activation='relu'),
        Dense(units=16, activation='relu'),
        Dense(units=8, activation='relu'),
        Dense(units=4, activation='relu'),
        Dense(units=1, activation='relu'),
    ]
)

model.compile(optimizer='Adagrad', loss='mae', metrics=['mae'])
X_train, X_test, Y_train, Y_test = tts(X, Y)

model.fit(
    x=X_train,
    y=Y_train,
    validation_data=(X_test, Y_test),
    batch_size=5,
    epochs=200
)

Epoch 1/200
10/10 [==============================] - 1s 70ms/step - loss: 196.4854 - mae: 196.4854 - val_loss: 111.9179 - val_mae: 111.9179
Epoch 2/200
10/10 [==============================] - 1s 56ms/step - loss: 72.6865 - mae: 72.6865 - val_loss: 27.7006 - val_mae: 27.7006
Epoch 3/200
10/10 [==============================] - 1s 53ms/step - loss: 48.5974 - mae: 48.5974 - val_loss: 74.3509 - val_mae: 74.3509
Epoch 4/200
10/10 [==============================] - 1s 53ms/step - loss: 37.0426 - mae: 37.0426 - val_loss: 43.4135 - val_mae: 43.4135
Epoch 5/200
10/10 [==============================] - 1s 53ms/step - loss: 38.0960 - mae: 38.0959 - val_loss: 56.3619 - val_mae: 56.3619
Epoch 6/200
10/10 [==============================] - 1s 53ms/step - loss: 32.6888 - mae: 32.6888 - val_loss: 28.0953 - val_mae: 28.0953
Epoch 7/200
10/10 [==============================] - 1s 55ms/step - loss: 39.1733 - mae: 39.1733 - val_loss: 84.9091 - val_mae: 84.9091
Epoch 8/200
10/10 [=========================